In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 2019 Part-time MSBA 6330 Final Exam, Part 2 - Danny Moncada

This workbook contains no actual questions. For actual questions, please go to [final exam questions](http://idsdl.csom.umn.edu/c/bigdata19/index.php?p=showsolution&key==Y2dz5yZrdTU24WTC1SbyVGdklWbc52bpRXds92c&after=5EDMy8SOw8COwACMwoDMwoDO) (user: guest ; password: 1234321). This exam questions is view only by design as I don't want to questions to be distributed. 

Feel free to open the exam questions on a different screen or a computer/mobile device.

## A. Using Spark RDD to process `movies.dat`

1\. [http://idsdl.csom.umn.edu/c/share/movies.dat](http://idsdl.csom.umn.edu/c/share/movies.dat)

In [2]:
! wget http://idsdl.csom.umn.edu/c/share/movies.dat

--2019-08-16 20:04:12--  http://idsdl.csom.umn.edu/c/share/movies.dat
Resolving idsdl.csom.umn.edu (idsdl.csom.umn.edu)... 134.84.138.46, 2607:ea00:101:480a:250:56ff:febb:e76b
Connecting to idsdl.csom.umn.edu (idsdl.csom.umn.edu)|134.84.138.46|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171308 (167K)
Saving to: ‘movies.dat’

movies.dat          100%[===================>] 167.29K  --.-KB/s    in 0.005s  

2019-08-16 20:04:12 (35.2 MB/s) - ‘movies.dat’ saved [171308/171308]



In [3]:
! head -10 movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller


2\.

In [8]:
## Create an RDD from the movies.dat file, split each line on :: since that is the delimiter as per looking at the file above.

movies = sc.textFile("movies.dat") \
    .map(lambda line: line.split("::"))

In [9]:
## Cache the RDD so that it doesn't read the data file again.

movies.cache()

PythonRDD[10] at RDD at PythonRDD.scala:53

3\. 

In [11]:
movies.collect()
movies.count()

3883

**Question**: Explain the role of cache().

**Answer**: `count` and `collect` are both "Action" operations on an RDD.  These return a local data set / value.  What caching does is makes it so that the file isn't read twice - it keeps the RDD in memory.  That being said, the two "Action" operations still read through the whole data frame twice, the first time when it "collects" and displays output, and the second time when it performs the "count" function to display how many rows are in the RDD.  Caching is nice if you had to filter and THEN do a count, since it would just read the RDD (that was already in memory) once.  However, in this instance, it still has to read the RDD twice so it's not as useful/effective.

4\. 

In [12]:
## Show the first ten rows of the RDD

movies.take(10)

[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"],
 ['2', 'Jumanji (1995)', "Adventure|Children's|Fantasy"],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama'],
 ['5', 'Father of the Bride Part II (1995)', 'Comedy'],
 ['6', 'Heat (1995)', 'Action|Crime|Thriller'],
 ['7', 'Sabrina (1995)', 'Comedy|Romance'],
 ['8', 'Tom and Huck (1995)', "Adventure|Children's"],
 ['9', 'Sudden Death (1995)', 'Action'],
 ['10', 'GoldenEye (1995)', 'Action|Adventure|Thriller']]

5\. 

In [13]:
import re
# the regular expression pattern has been given to you.
pattern = re.compile("(.*)\((\d+)\)$")

def parse_nameyear(s):
    ''' parse the string s and return [name,year]. In case of error, returns [None, None]
        e.g. parse_nameyear("Tom and Huck (1995)") returns ["Tom and Huck", "1995"]
    '''
    matched = pattern.match(s)
    if matched:
        return [matched.group(1).strip(),int(matched.group(2))]
    else:
        return [None, None]

In [22]:
## Create a function to get the elements that we want, movieid, movie name, year
## Creates a tuple with movieid (as int), movie name, year (as int)

def movie_year(line):
    return int(line[0]), parse_nameyear(line[1])[0], int(parse_nameyear(line[1])[1])

In [24]:
## Create an RDD movieyear from movies using my function, that gets me movieid, movie name, year

movieyear = movies.map(lambda r: movie_year(r))

In [25]:
## Verify the content of the RDD

movieyear.take(5)

[(1, 'Toy Story', 1995),
 (2, 'Jumanji', 1995),
 (3, 'Grumpier Old Men', 1995),
 (4, 'Waiting to Exhale', 1995),
 (5, 'Father of the Bride Part II', 1995)]

6\. 

In [32]:
## Join the three elements of each row in the RDD with a \t , then save this as the output

movie_year_tab = movieyear.map(lambda r: "\t".join(map(str, r)))

In [33]:
movie_year_tab.saveAsTextFile("movieyear")

7\. 

In [37]:
## Look at the output from saving the movieyear file - it makes it a directory

! ls -l movieyear/

total 108
-rw-rw-r-- 1 vagrant vagrant 108542 Aug 16 20:24 part-00000
-rw-rw-r-- 1 vagrant vagrant      0 Aug 16 20:24 _SUCCESS


In [36]:
## Confirm the first ten rows of the file - looks good!

! cat movieyear/* | head -10

1	Toy Story	1995
2	Jumanji	1995
3	Grumpier Old Men	1995
4	Waiting to Exhale	1995
5	Father of the Bride Part II	1995
6	Heat	1995
7	Sabrina	1995
8	Tom and Huck	1995
9	Sudden Death	1995
10	GoldenEye	1995
cat: write error: Broken pipe


## B. Use Spark SQL to Analyze `SFPD` dataset

8\. [http://idsdl.csom.umn.edu/c/share/sfpd.json](http://idsdl.csom.umn.edu/c/share/sfpd.json)

In [38]:
## Download the file

! wget http://idsdl.csom.umn.edu/c/share/sfpd.json

--2019-08-16 20:27:37--  http://idsdl.csom.umn.edu/c/share/sfpd.json
Resolving idsdl.csom.umn.edu (idsdl.csom.umn.edu)... 134.84.138.46, 2607:ea00:101:480a:250:56ff:febb:e76b
Connecting to idsdl.csom.umn.edu (idsdl.csom.umn.edu)|134.84.138.46|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1863319 (1.8M) [application/json]
Saving to: ‘sfpd.json’

sfpd.json           100%[===================>]   1.78M  --.-KB/s    in 0.1s    

2019-08-16 20:27:37 (15.3 MB/s) - ‘sfpd.json’ saved [1863319/1863319]



In [39]:
## Examine the first 5 rows

! head -5 sfpd.json

{"Category": "FRAUD", "Descript": "CREDIT CARD, THEFT BY USE OF", "PdDistrict": "CENTRAL", "Location": "(37.7878271137225, -122.408384275542)", "IncidentNum": "150446227", "Time": "23:59", "Y": 37.7878271137225, "X": -122.408384275542, "Resolution": "NONE", "DayOfWeek": "Thursday", "Address": "300 Block of POWELL ST", "Date": "05/21/2015", "PdId": 15044622709320}
{"Category": "NON-CRIMINAL", "Descript": "LOST PROPERTY", "PdDistrict": "CENTRAL", "Location": "(37.7878271137225, -122.408384275542)", "IncidentNum": "150446227", "Time": "23:59", "Y": 37.7878271137225, "X": -122.408384275542, "Resolution": "NONE", "DayOfWeek": "Thursday", "Address": "300 Block of POWELL ST", "Date": "05/21/2015", "PdId": 15044622771000}
{"Category": "WARRANTS", "Descript": "ENROUTE TO OUTSIDE JURISDICTION", "PdDistrict": "CENTRAL", "Location": "(37.8082613305439, -122.414988252591)", "IncidentNum": "150444550", "Time": "23:54", "Y": 37.8082613305439, "X": -122.414988252591, "Resolution": "ARREST, BOOKED", "D

9\.

In [40]:
## Use spark json reader to read in the file

sfpd = spark.read.json("sfpd.json")

In [41]:
## Examine the DataFrame schema to make sure it looks good - per the data types in the instructions, everything is a string except for X and Y, we'll leave them as DOUBLE

sfpd.printSchema()

root
 |-- Address: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Descript: string (nullable = true)
 |-- IncidentNum: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- PdDistrict: string (nullable = true)
 |-- PdId: long (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)



In [43]:
## Limit to 5 rows so we don't read in the entire DataFrame, then use toPandas to make it readable.  Looks pretty clean to me!

sfpd.limit(5).toPandas()

,Address,Category,Date,DayOfWeek,Descript,IncidentNum,Location,PdDistrict,PdId,Resolution,Time,X,Y
0,300 Block of POWELL ST,FRAUD,05/21/2015,Thursday,"CREDIT CARD, THEFT BY USE OF",150446227,"(37.7878271137225, -122.408384275542)",CENTRAL,15044622709320,NONE,23:59,-122.408384,37.787827
1,300 Block of POWELL ST,NON-CRIMINAL,05/21/2015,Thursday,LOST PROPERTY,150446227,"(37.7878271137225, -122.408384275542)",CENTRAL,15044622771000,NONE,23:59,-122.408384,37.787827
2,100 Block of JEFFERSON ST,WARRANTS,05/21/2015,Thursday,ENROUTE TO OUTSIDE JURISDICTION,150444550,"(37.8082613305439, -122.414988252591)",CENTRAL,15044455062050,"ARREST, BOOKED",23:54,-122.414988,37.808261
3,100 Block of JEFFERSON ST,WARRANTS,05/21/2015,Thursday,WARRANT ARREST,150444550,"(37.8082613305439, -122.414988252591)",CENTRAL,15044455063010,"ARREST, BOOKED",23:54,-122.414988,37.808261
4,100 Block of JEFFERSON ST,OTHER OFFENSES,05/21/2015,Thursday,TRAFFIC VIOLATION ARREST,150444550,"(37.8082613305439, -122.414988252591)",CENTRAL,15044455065010,"ARREST, BOOKED",23:54,-122.414988,37.808261


10\.

In [44]:
## Get summary statistics to make sure all elements were loaded successfully

sfpd.describe().toPandas()

,summary,Address,Category,Date,DayOfWeek,Descript,IncidentNum,Location,PdDistrict,PdId,Resolution,Time,X,Y
0,count,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
1,mean,None,None,None,None,None,1.513429158822E8,None,None,1.5134291615212957E13,None,None,-122.42551042937846,37.767211340211325
2,stddev,None,None,None,None,None,2136698.504698914,None,None,2.136698449241794E11,None,None,0.027885011741052634,0.02465335189943103
3,min,0 Block of 12TH ST,ARSON,05/09/2015,Friday,"ACCESS CARD INFORMATION, THEFT OF",130525465,"(37.7084449580792, -122.458822989017)",BAYVIEW,13052546507023,"ARREST, BOOKED",00:01,-122.513642064265,37.7084449580792
4,max,ZOE ST / BRANNAN ST,WEAPON LAWS,05/21/2015,Wednesday,"WEAPON, POSSESS OR BRING OTHER ON SCHOOL GROUNDS",159002606,"(37.819975492297, -122.37427517671)",TENDERLOIN,15900260627170,UNFOUNDED,23:59,-122.365565425353,37.819975492297


-

**Answer**: There are NO missing values, per the summary statistics.  Every column has 5000 in the count, that means that the entire dataset made it over!

11\. 

In [46]:
## Exclude these columns [Descript, Address, Location, PdId, IncidentNum]
## Create a new DF sfpd2 with just the columns we want

sfpd2 = sfpd.select("Category", "Date", "DayOfWeek", "PdDistrict", "Resolution", "Time", "X", "Y")

In [47]:
## Just check to make sure that new DF was created succesfully

sfpd2.limit(5).toPandas()

,Category,Date,DayOfWeek,PdDistrict,Resolution,Time,X,Y
0,FRAUD,05/21/2015,Thursday,CENTRAL,NONE,23:59,-122.408384,37.787827
1,NON-CRIMINAL,05/21/2015,Thursday,CENTRAL,NONE,23:59,-122.408384,37.787827
2,WARRANTS,05/21/2015,Thursday,CENTRAL,"ARREST, BOOKED",23:54,-122.414988,37.808261
3,WARRANTS,05/21/2015,Thursday,CENTRAL,"ARREST, BOOKED",23:54,-122.414988,37.808261
4,OTHER OFFENSES,05/21/2015,Thursday,CENTRAL,"ARREST, BOOKED",23:54,-122.414988,37.808261


12\. 

In [52]:
## Group by the DayOfWeek
# Do a count to get the count of days
# Rename the column for clarity
# Order by count of days descending and show the result

day_of_week_freq = sfpd2.groupBy("DayOfWeek").count().withColumnRenamed("count", "count_of_days").orderBy("count_of_days", ascending = False)

In [53]:
day_of_week_freq.show()

+---------+-------------+
|DayOfWeek|count_of_days|
+---------+-------------+
|   Monday|          788|
|   Sunday|          776|
| Saturday|          774|
| Thursday|          757|
|  Tuesday|          755|
|Wednesday|          723|
|   Friday|          427|
+---------+-------------+



13\. 

In [55]:
## Explore the field Resolution and determine if there are multiple values in the column, just get a distinct set of values for the column

sfpd2.select("Resolution").distinct().show(truncate = False)

+---------------------+
|Resolution           |
+---------------------+
|JUVENILE BOOKED      |
|EXCEPTIONAL CLEARANCE|
|ARREST, BOOKED       |
|LOCATED              |
|UNFOUNDED            |
|PSYCHOPATHIC CASE    |
|ARREST, CITED        |
|NONE                 |
+---------------------+



**Answer**: Yes, there are *two* instances of multiple values: ARREST, BOOKED and ARREST, CITED.

Unrelated, but there is a resolution code for PYSCHOPATHIC CASE - YIKES!

14\.

In [57]:
## Import sql functions for the manipulation below, and then much later on in the notebook

import pyspark.sql.functions as f

In [67]:
## A lot in this step, so we take it one at a time
## The first line creates a new column called "resolution", where I split the "Resolution" column on the ", " so I can split the ARREST/BOOKED and ARREST/CITED into two buckets
## Explode them so that there is a row for each - so that we don't mess up the count
## Group them by resolution, our newly renamed column
## Peform a count
## Rename the count column to num_case
## Order by num_case in descending order


case_freq = sfpd2.withColumn("resolution", f.explode(f.split(sfpd2.Resolution, ", "))) \
        .groupBy("resolution") \
        .count() \
        .withColumnRenamed("count", "num_case") \
        .orderBy("num_case", ascending = False)

In [68]:
## Show the result as a Pandas DataFrame for readability

case_freq.toPandas()

,resolution,num_case
0,NONE,3642
1,ARREST,1221
2,BOOKED,1179
3,UNFOUNDED,60
4,CITED,42
5,JUVENILE BOOKED,36
6,LOCATED,19
7,EXCEPTIONAL CLEARANCE,14
8,PSYCHOPATHIC CASE,8


15\.

In [69]:
## Create a temporary view to do some SQL magic!

sfpd2.createOrReplaceTempView("category_pdDistrict_view")

In [80]:
## Create a new DataFrame based on the SQL query.  We select the category, the pdDistrit, and a count for the number of cases, group them by the category then pdDistrict
## HAVING filters out on those combinations with 30+ cases
## Order the results in DESC order

category_pdDistrict_analysis = \
    spark.sql("SELECT Category as category, PdDistrict as pdDistrict, count(*) as num_cases FROM category_pdDistrict_view GROUP BY category, pdDistrict HAVING num_cases >= 30 ORDER BY num_cases DESC")

In [140]:
## Show the entire table - I did not convert to a Pandas DataFrame because this is a simple table.

category_pdDistrict_analysis.show(100)

+--------------+----------+---------+
|      category|pdDistrict|num_cases|
+--------------+----------+---------+
| LARCENY/THEFT|  SOUTHERN|      315|
| LARCENY/THEFT|  NORTHERN|      249|
| LARCENY/THEFT|   CENTRAL|      244|
| LARCENY/THEFT|   TARAVAL|      139|
|  NON-CRIMINAL|  SOUTHERN|      110|
| LARCENY/THEFT|      PARK|      104|
|OTHER OFFENSES|   MISSION|       97|
| LARCENY/THEFT|  RICHMOND|       94|
|OTHER OFFENSES|  SOUTHERN|       91|
|OTHER OFFENSES|   BAYVIEW|       90|
|OTHER OFFENSES| INGLESIDE|       83|
| LARCENY/THEFT|   MISSION|       79|
|  NON-CRIMINAL|   TARAVAL|       69|
|  NON-CRIMINAL|  NORTHERN|       69|
|       ASSAULT|   BAYVIEW|       69|
|  NON-CRIMINAL|   CENTRAL|       69|
| LARCENY/THEFT|   BAYVIEW|       68|
| LARCENY/THEFT| INGLESIDE|       68|
|  NON-CRIMINAL|   MISSION|       66|
|       ASSAULT|   MISSION|       65|
|       ASSAULT|  SOUTHERN|       62|
|OTHER OFFENSES|  NORTHERN|       60|
| LARCENY/THEFT|TENDERLOIN|       59|
|OTHER OFFEN

16\.

In [82]:
## Save sfpd2 dataframe as parquet - no need to change anything to save it as parquet - this is default behavior.  Overwrite just in case I have to do it again.

sfpd2.write.mode("overwrite").save("sfpd2")

In [83]:
! ls -l sfpd2/

total 80
-rw-rw-r-- 1 vagrant vagrant 79627 Aug 16 21:08 part-00000-dd98bd50-e316-41c7-b535-d20cf76c3cb3-c000.snappy.parquet
-rw-rw-r-- 1 vagrant vagrant     0 Aug 16 21:08 _SUCCESS


In [85]:
## Parquet format - so unreadable to the human eye, but I just want to see that it did write data

! cat sfpd2/* | head -1

UNDER!$ INFLUENCE@PBRIBERY
cat: write error: Broken pipe


## C. Using MLlib to do Churn prediction

In [138]:
## import the libraries for doing MLLib

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

17\. [http://idsdl.csom.umn.edu/c/share/train.csv](http://idsdl.csom.umn.edu/c/share/train.csv) and  [http://idsdl.csom.umn.edu/c/share/test.csv](http://idsdl.csom.umn.edu/c/share/test.csv)

In [87]:
! wget http://idsdl.csom.umn.edu/c/share/train.csv

--2019-08-16 21:12:00--  http://idsdl.csom.umn.edu/c/share/train.csv
Resolving idsdl.csom.umn.edu (idsdl.csom.umn.edu)... 134.84.138.46, 2607:ea00:101:480a:250:56ff:febb:e76b
Connecting to idsdl.csom.umn.edu (idsdl.csom.umn.edu)|134.84.138.46|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141010 (138K) [text/csv]
Saving to: ‘train.csv’

train.csv           100%[===================>] 137.71K  --.-KB/s    in 0.003s  

2019-08-16 21:12:00 (43.5 MB/s) - ‘train.csv’ saved [141010/141010]



In [88]:
! wget http://idsdl.csom.umn.edu/c/share/test.csv

--2019-08-16 21:12:09--  http://idsdl.csom.umn.edu/c/share/test.csv
Resolving idsdl.csom.umn.edu (idsdl.csom.umn.edu)... 134.84.138.46, 2607:ea00:101:480a:250:56ff:febb:e76b
Connecting to idsdl.csom.umn.edu (idsdl.csom.umn.edu)|134.84.138.46|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35493 (35K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>]  34.66K  --.-KB/s    in 0.003s  

2019-08-16 21:12:09 (12.7 MB/s) - ‘test.csv’ saved [35493/35493]



18\.

In [89]:
## Examine first 10 lines to verify format, looks like this will be a CSV, straight forward.  Has a column header already which is nice.

! head -10 *.csv

==> test.csv <==
Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total eve minutes,Total eve calls,Total night minutes,Total night calls,Total intl minutes,Total intl calls,Customer service calls,Churn
117,0.0,0.0,0,184.5,97,351.6,80,215.8,90,8.7,4,1,0
65,0.0,0.0,0,129.1,137,228.5,83,208.8,111,12.7,6,4,1
161,0.0,0.0,0,332.9,67,317.8,97,160.6,128,5.4,9,4,1
111,0.0,0.0,0,110.4,103,137.3,102,189.6,105,7.7,6,2,0
49,0.0,0.0,0,119.3,117,215.1,109,178.7,90,11.1,1,1,0
36,0.0,1.0,30,146.3,128,162.5,80,129.3,109,14.5,6,0,0
65,0.0,0.0,0,211.3,120,162.6,122,134.7,118,13.2,5,3,0
119,0.0,0.0,0,159.1,114,231.3,117,143.2,91,8.8,3,5,1
10,0.0,0.0,0,186.1,112,190.2,66,282.8,57,11.4,6,2,0

==> train.csv <==
Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total eve minutes,Total eve calls,Total night minutes,Total night calls,Total intl minutes,Total intl calls,Customer service calls,Churn
12

19\.

In [91]:
## Schema will probably be OK as well - doesn't look like a messy file
## Load the CSVs into new DataFrames, traindata and testdata respectively.


traindata = sqlCtx.read.format("csv").option("header", "true") \
    .option("inferSchema", "true").load("train.csv")

testdata = sqlCtx.read.format("csv").option("header", "true") \
    .option("inferSchema", "true").load("test.csv")

In [94]:
## show the DataFrame schema to confirm load was successfully, and then show the first 5 rows.  Looks good!

traindata.printSchema()

traindata.limit(5).toPandas()

root
 |-- Account length: integer (nullable = true)
 |-- International plan: double (nullable = true)
 |-- Voice mail plan: double (nullable = true)
 |-- Number vmail messages: integer (nullable = true)
 |-- Total day minutes: double (nullable = true)
 |-- Total day calls: integer (nullable = true)
 |-- Total eve minutes: double (nullable = true)
 |-- Total eve calls: integer (nullable = true)
 |-- Total night minutes: double (nullable = true)
 |-- Total night calls: integer (nullable = true)
 |-- Total intl minutes: double (nullable = true)
 |-- Total intl calls: integer (nullable = true)
 |-- Customer service calls: integer (nullable = true)
 |-- Churn: integer (nullable = true)



,Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total eve minutes,Total eve calls,Total night minutes,Total night calls,Total intl minutes,Total intl calls,Customer service calls,Churn
0,128,0.0,1.0,25,265.1,110,197.4,99,244.7,91,10.0,3,1,0
1,107,0.0,1.0,26,161.6,123,195.5,103,254.4,103,13.7,3,1,0
2,137,0.0,0.0,0,243.4,114,121.2,110,162.6,104,12.2,5,0,0
3,84,1.0,0.0,0,299.4,71,61.9,88,196.9,89,6.6,7,2,0
4,75,1.0,0.0,0,166.7,113,148.3,122,186.9,121,10.1,3,3,0


In [95]:
## show the DataFrame schema to confirm load was successfully, and then show the first 5 rows.  Looks good!

testdata.printSchema()

testdata.limit(5).toPandas()

root
 |-- Account length: integer (nullable = true)
 |-- International plan: double (nullable = true)
 |-- Voice mail plan: double (nullable = true)
 |-- Number vmail messages: integer (nullable = true)
 |-- Total day minutes: double (nullable = true)
 |-- Total day calls: integer (nullable = true)
 |-- Total eve minutes: double (nullable = true)
 |-- Total eve calls: integer (nullable = true)
 |-- Total night minutes: double (nullable = true)
 |-- Total night calls: integer (nullable = true)
 |-- Total intl minutes: double (nullable = true)
 |-- Total intl calls: integer (nullable = true)
 |-- Customer service calls: integer (nullable = true)
 |-- Churn: integer (nullable = true)



,Account length,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total eve minutes,Total eve calls,Total night minutes,Total night calls,Total intl minutes,Total intl calls,Customer service calls,Churn
0,117,0.0,0.0,0,184.5,97,351.6,80,215.8,90,8.7,4,1,0
1,65,0.0,0.0,0,129.1,137,228.5,83,208.8,111,12.7,6,4,1
2,161,0.0,0.0,0,332.9,67,317.8,97,160.6,128,5.4,9,4,1
3,111,0.0,0.0,0,110.4,103,137.3,102,189.6,105,7.7,6,2,0
4,49,0.0,0.0,0,119.3,117,215.1,109,178.7,90,11.1,1,1,0


In [96]:
## Cache the dataframes because we'll be doing a lot of manipulation it seems!

traindata.cache()
testdata.cache()

DataFrame[Account length: int, International plan: double, Voice mail plan: double, Number vmail messages: int, Total day minutes: double, Total day calls: int, Total eve minutes: double, Total eve calls: int, Total night minutes: double, Total night calls: int, Total intl minutes: double, Total intl calls: int, Customer service calls: int, Churn: int]

20\. 

In [99]:
## Create a function to remove white space " " from column headers and replace with "_" - to save time and make it reusable.

def remove_whitespace(df):
    for col in df.columns:
        df = df.withColumnRenamed(col, col.replace(" ", "_"))
    return df

In [102]:
## Implement the function to remove white space and replace with _

traindata = remove_whitespace(traindata)

testdata = remove_whitespace(testdata)

In [105]:
## Covert all of the column types to double - we can use another function to do this programmatically
## function takes each column in the dataframe and just CASTs it as a DOUBLE.

def double_it(df):
    for col in df.columns:
        df = df.withColumn(
        col, 
        f.col(col).cast("double")
        )
    return df

In [108]:
## test function to make sure it works

double_it(traindata)

DataFrame[Account_length: double, International_plan: double, Voice_mail_plan: double, Number_vmail_messages: double, Total_day_minutes: double, Total_day_calls: double, Total_eve_minutes: double, Total_eve_calls: double, Total_night_minutes: double, Total_night_calls: double, Total_intl_minutes: double, Total_intl_calls: double, Customer_service_calls: double, Churn: double]

In [109]:
## Make everything DOUBLE datatype and overwrite original dataframes

traindata = double_it(traindata)

testdata = double_it(testdata)

In [110]:
## Confirm transformations were successful - We look good!

traindata.printSchema()

testdata.printSchema()

root
 |-- Account_length: double (nullable = true)
 |-- International_plan: double (nullable = true)
 |-- Voice_mail_plan: double (nullable = true)
 |-- Number_vmail_messages: double (nullable = true)
 |-- Total_day_minutes: double (nullable = true)
 |-- Total_day_calls: double (nullable = true)
 |-- Total_eve_minutes: double (nullable = true)
 |-- Total_eve_calls: double (nullable = true)
 |-- Total_night_minutes: double (nullable = true)
 |-- Total_night_calls: double (nullable = true)
 |-- Total_intl_minutes: double (nullable = true)
 |-- Total_intl_calls: double (nullable = true)
 |-- Customer_service_calls: double (nullable = true)
 |-- Churn: double (nullable = true)

root
 |-- Account_length: double (nullable = true)
 |-- International_plan: double (nullable = true)
 |-- Voice_mail_plan: double (nullable = true)
 |-- Number_vmail_messages: double (nullable = true)
 |-- Total_day_minutes: double (nullable = true)
 |-- Total_day_calls: double (nullable = true)
 |-- Total_eve_minut

In [115]:
## Determine the correct feature columns - per the instructions it should be everything BUT Churn
## Easily done by subscripting the DataFrames columns to only return the first to the second to last column
## Set featureCols to this output

featureCols = traindata.columns[0:-1]

## Build my VectorAssembler and create my features column

va = VectorAssembler(inputCols = featureCols, outputCol = "features")

## Just double checking my features columns are correct
featureCols

['Account_length',
 'International_plan',
 'Voice_mail_plan',
 'Number_vmail_messages',
 'Total_day_minutes',
 'Total_day_calls',
 'Total_eve_minutes',
 'Total_eve_calls',
 'Total_night_minutes',
 'Total_night_calls',
 'Total_intl_minutes',
 'Total_intl_calls',
 'Customer_service_calls']

In [118]:
## Testing to make sure VectorAssembler works
## Show the first 8 rows to confirm transformation will work

vectorized = va.transform(testdata)

vectorized.select("features").show(10, truncate = False)

+-------------------------------------------------------------------+
|features                                                           |
+-------------------------------------------------------------------+
|[117.0,0.0,0.0,0.0,184.5,97.0,351.6,80.0,215.8,90.0,8.7,4.0,1.0]   |
|[65.0,0.0,0.0,0.0,129.1,137.0,228.5,83.0,208.8,111.0,12.7,6.0,4.0] |
|[161.0,0.0,0.0,0.0,332.9,67.0,317.8,97.0,160.6,128.0,5.4,9.0,4.0]  |
|[111.0,0.0,0.0,0.0,110.4,103.0,137.3,102.0,189.6,105.0,7.7,6.0,2.0]|
|[49.0,0.0,0.0,0.0,119.3,117.0,215.1,109.0,178.7,90.0,11.1,1.0,1.0] |
|[36.0,0.0,1.0,30.0,146.3,128.0,162.5,80.0,129.3,109.0,14.5,6.0,0.0]|
|[65.0,0.0,0.0,0.0,211.3,120.0,162.6,122.0,134.7,118.0,13.2,5.0,3.0]|
|[119.0,0.0,0.0,0.0,159.1,114.0,231.3,117.0,143.2,91.0,8.8,3.0,5.0] |
|[10.0,0.0,0.0,0.0,186.1,112.0,190.2,66.0,282.8,57.0,11.4,6.0,2.0]  |
|[68.0,0.0,0.0,0.0,148.8,70.0,246.5,164.0,129.8,103.0,12.1,3.0,3.0] |
+-------------------------------------------------------------------+
only showing top 10 

In [119]:
## Build the LR model using my features column and my indexed Survived column

lr = LogisticRegression(featuresCol = "features", labelCol = "Churn")

In [120]:
## Build my pipeline and get ready to feed it to my LR model

pl = Pipeline(stages = [va, lr])

In [121]:
## Run my pipeline to test the LogisticRegressionModel to make sure it works

pipelineModel = pl.fit(traindata)

## Transform my testdata to see if it looks ok

predictions = pipelineModel.transform(testdata)

In [125]:
## Show the first 5 rows

predictions.select("Churn", "features", "prediction").limit(10).toPandas()

,Churn,features,prediction
0,0.0,"[117.0, 0.0, 0.0, 0.0, 184.5, 97.0, 351.6, 80....",0.0
1,1.0,"[65.0, 0.0, 0.0, 0.0, 129.1, 137.0, 228.5, 83....",0.0
2,1.0,"[161.0, 0.0, 0.0, 0.0, 332.9, 67.0, 317.8, 97....",1.0
3,0.0,"[111.0, 0.0, 0.0, 0.0, 110.4, 103.0, 137.3, 10...",0.0
4,0.0,"[49.0, 0.0, 0.0, 0.0, 119.3, 117.0, 215.1, 109...",0.0
5,0.0,"[36.0, 0.0, 1.0, 30.0, 146.3, 128.0, 162.5, 80...",0.0
6,0.0,"[65.0, 0.0, 0.0, 0.0, 211.3, 120.0, 162.6, 122...",0.0
7,1.0,"[119.0, 0.0, 0.0, 0.0, 159.1, 114.0, 231.3, 11...",0.0
8,0.0,"[10.0, 0.0, 0.0, 0.0, 186.1, 112.0, 190.2, 66....",0.0
9,0.0,"[68.0, 0.0, 0.0, 0.0, 148.8, 70.0, 246.5, 164....",0.0


21\. 

In [127]:
## Create the ParamGrid to test regParam three levels.
paramGrid = ParamGridBuilder().addGrid(lr.regParam,[0.0,1.0,2.0]).build()

## Build an evaluator to test the RForest Classifier using areaUnderROC as the test metric
evaluator = BinaryClassificationEvaluator(labelCol = 'Churn', rawPredictionCol = 'prediction', metricName='areaUnderROC')

## Build the cross validator based on the RForest Pipeline above, the ParamGrid, the evaluation metrics, and 3 folds
crossval = CrossValidator(estimator = pl, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = 3)

22\.

In [129]:
## Run the CrossValidator on the train dataset and create a new output churn_model
churn_model = crossval.fit(traindata)

23\.

In [130]:
## Transform the test data set WITH THE BEST MODEL and try to do a better job predicting!

transformed_data = churn_model.bestModel.transform(testdata)

In [136]:
## Present the first 10 rows of the prediction outcome DF with churn, prediction and ALLLLLLLLLLLLLL feature columns

transformed_data.select("Churn", "prediction", "Account_length", \
                        "International_plan", "Voice_mail_plan", "Number_vmail_messages", "Total_day_minutes", \
                        "Total_eve_minutes", "Total_eve_calls", "Total_night_minutes", "Total_night_calls", \
                        "Total_intl_minutes", "Total_intl_calls", "Customer_service_calls").limit(10).toPandas()

,Churn,prediction,Account_length,International_plan,Voice_mail_plan,Number_vmail_messages,Total_day_minutes,Total_eve_minutes,Total_eve_calls,Total_night_minutes,Total_night_calls,Total_intl_minutes,Total_intl_calls,Customer_service_calls
0,0.0,0.0,117.0,0.0,0.0,0.0,184.5,351.6,80.0,215.8,90.0,8.7,4.0,1.0
1,1.0,0.0,65.0,0.0,0.0,0.0,129.1,228.5,83.0,208.8,111.0,12.7,6.0,4.0
2,1.0,1.0,161.0,0.0,0.0,0.0,332.9,317.8,97.0,160.6,128.0,5.4,9.0,4.0
3,0.0,0.0,111.0,0.0,0.0,0.0,110.4,137.3,102.0,189.6,105.0,7.7,6.0,2.0
4,0.0,0.0,49.0,0.0,0.0,0.0,119.3,215.1,109.0,178.7,90.0,11.1,1.0,1.0
5,0.0,0.0,36.0,0.0,1.0,30.0,146.3,162.5,80.0,129.3,109.0,14.5,6.0,0.0
6,0.0,0.0,65.0,0.0,0.0,0.0,211.3,162.6,122.0,134.7,118.0,13.2,5.0,3.0
7,1.0,0.0,119.0,0.0,0.0,0.0,159.1,231.3,117.0,143.2,91.0,8.8,3.0,5.0
8,0.0,0.0,10.0,0.0,0.0,0.0,186.1,190.2,66.0,282.8,57.0,11.4,6.0,2.0
9,0.0,0.0,68.0,0.0,0.0,0.0,148.8,246.5,164.0,129.8,103.0,12.1,3.0,3.0


24\. 

In [137]:
AoC_eval = BinaryClassificationEvaluator(labelCol = "Churn", rawPredictionCol = "prediction", metricName = "areaUnderROC")
AoC = AoC_eval.evaluate(transformed_data)
print("AreaUnderROC = %s" % AoC)

AreaUnderROC = 0.5772543246227456


**Answer**: The area under the ROC metrics for the predictions obtained for the test dataset is 0.5772543246227456.  Yikes!

# Professor De, thank you very much for a great semester.  It has been a pleasure.  - DM


## Submission Check list

- Convert the jupyter notebook to a single **html file** and submit the html file. 
- You have listed the question numbers clearly for grading purpose.
- Make sure that when there is a bold-faced **Question**, you have an explicit **Answer**.